In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
df_web = pd.read_csv("../Data/Processed/df_final_web_merged.csv")

## Experiment Metrics /  KPI Dataframe

In [3]:
df_web.columns.tolist()

['client_id', 'visitor_id', 'visit_id', 'process_step', 'date', 'time']

In [4]:
df_web.head()

,client_id,visitor_id,visit_id,process_step,date,time
0,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12,20:19:36
1,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12,20:19:45
2,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12,20:20:31
3,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12,20:22:05
4,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12,20:23:09


In [5]:
# Working with a copy to avoid data loss
df = df_web.copy()

# building one column for datetime
df["datetime"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"].astype(str))

group_cols = ["client_id", "visitor_id", "visit_id"]

# sort events by visit date (and time)
df = df.sort_values(group_cols + ["datetime"])

# assigning step order for identifying backtracking
step_order_map = {
    "start": 0,
    "step_1": 1,
    "step_2": 2,
    "step_3": 3,
    "confirm": 4
}

df["step_order"] = df["process_step"].map(step_order_map)

# visit level summary for each client
summary = (
    df.groupby(group_cols)
      .agg(
          steps_count=("process_step", "count"),
          unique_steps=("process_step", "nunique"),
          start_time=("datetime", "min"),
          end_time=("datetime", "max")
      )
      .reset_index()
)

# completion (binary values)
completed_by_visit = (
    df.groupby(group_cols)["process_step"]
      .apply(lambda x: int("confirm" in x.values))
      .reset_index(name="completion_status")
)

summary = summary.merge(completed_by_visit, on=group_cols, how="left")

# drop-off step (if experiment not completed)
last_step = (
    df.groupby(group_cols)["process_step"]
      .last()
      .reset_index(name="last_step")
)

summary = summary.merge(last_step, on=group_cols, how="left")

summary["drop_off_step"] = np.where(
    summary["completion_status"] == 1,
    None,
    summary["last_step"]
)

summary.drop(columns="last_step", inplace=True)

# most frequent step (happens during backtracks)
most_freq = (
    df.groupby(group_cols)["process_step"]
      .agg(lambda x: x.value_counts().idxmax() if x.value_counts().max() > 1 else "none")
      .reset_index(name="most_frequent_step")
)

summary = summary.merge(most_freq, on=group_cols, how="left")

# total time and average time per step
summary["total_seconds"] = (summary["end_time"] - summary["start_time"]).dt.total_seconds()
summary["avg_time_per_step_sec"] = summary["total_seconds"] / summary["steps_count"]

summary["avg_time_per_step"] = summary["avg_time_per_step_sec"].apply(
    lambda x: f"{int(x//60)}:{int(x%60):02d}" if pd.notna(x) else Noneone
)

# tracking backtracks
df["prev_step_order"] = df.groupby(group_cols)["step_order"].shift(1)
df["is_backtrack"] = ((df["step_order"] < df["prev_step_order"]) & df["step_order"].notna() & df["prev_step_order"].notna()).astype(int)

backtracks = (
    df.groupby(group_cols)["is_backtrack"]
      .agg(backtrack_count="sum")
      .reset_index()
)

backtracks["backtrack_flag"] = (backtracks["backtrack_count"] > 0).astype(int)

summary = summary.merge(backtracks, on=group_cols, how="left")


# Time to next step
df["next_datetime"] = df.groupby(group_cols)["datetime"].shift(-1)
df["delta_sec"] = (df["next_datetime"] - df["datetime"]).dt.total_seconds()

# Remove invalid time stamps
df.loc[df["delta_sec"] < 0, "delta_sec"] = np.nan

step_time = (
    df.groupby(group_cols + ["process_step"])["delta_sec"]
      .sum(min_count=1)
      .reset_index()
)

# Pivot - one column per step containing seconds spent in that particular step for the visit
step_time_wide = step_time.pivot_table(
    index=group_cols,
    columns="process_step",
    values="delta_sec",
    aggfunc="sum"
).reset_index()

# renaming columns
step_cols = [c for c in step_time_wide.columns if c not in group_cols]
step_time_wide = step_time_wide.rename(columns={c: f"time_in_{c}_sec" for c in step_cols})

summary = summary.merge(step_time_wide, on=group_cols, how="left")

# date / time fields
summary["date"] = summary["end_time"].dt.date

# Column order
df_summary = summary[[
    "client_id", "visitor_id", "visit_id",
    "steps_count", "unique_steps",
    "date", 
    "completion_status",          # binary 1/0
    "drop_off_step",
    "most_frequent_step",
    "avg_time_per_step",        
    "backtrack_flag",             # binary 1/0
    "backtrack_count",            # count of backward moves
    *[c for c in summary.columns if c.startswith("time_in_") and c.endswith("_sec")]  # time per step for all steps
]]


### Creating other total_time columns

In [6]:
df_summary = df_summary.copy()

# identify all per-step time columns
time_cols = [c for c in df_summary.columns if c.startswith("time_in_") and c.endswith("_sec")]

# total time in seconds
df_summary["total_time_sec"] = df_summary[time_cols].sum(axis=1, min_count=1)

# keep total time ONLY for completed visits
df_summary.loc[df_summary["completion_status"] != 1, "total_time_sec"] = np.nan

# time in mm:ss format
df_summary["total_time"] = df_summary["total_time_sec"].apply(
    lambda s: f"{int(s//60)}:{int(s%60):02d}" if pd.notna(s) else None
)

In [7]:
df_summary.to_csv(
    "../Data/Processed/df_web_visit_metrics_final.csv",
    index=False
)

In [8]:
df_summary.head()

,client_id,visitor_id,visit_id,steps_count,unique_steps,date,completion_status,drop_off_step,most_frequent_step,avg_time_per_step,backtrack_flag,backtrack_count,time_in_confirm_sec,time_in_start_sec,time_in_step_1_sec,time_in_step_2_sec,time_in_step_3_sec,total_time_sec,total_time
0,169,201385055_71273495308,749567106_99161211863_557568,5,5,2017-04-12,1,None,none,0:42,0,0,0.0,9.0,46.0,94.0,64.0,213.0,3:33
1,336,64757908_3400128256,649044751_80905125055_554468,2,1,2017-06-01,0,start,start,7:54,0,0,NaN,948.0,NaN,NaN,NaN,NaN,None
2,546,475037402_89828530214,731811517_9330176838_94847,5,5,2017-06-17,1,None,none,0:26,0,0,0.0,10.0,13.0,87.0,23.0,133.0,2:13
3,555,402506806_56087378777,637149525_38041617439_716659,5,5,2017-04-15,1,None,none,0:31,0,0,0.0,7.0,32.0,99.0,20.0,158.0,2:38
4,647,66758770_53988066587,40369564_40101682850_311847,5,5,2017-04-12,1,None,none,1:15,0,0,0.0,7.0,18.0,189.0,163.0,377.0,6:17


----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------

## Merging Client Demographics and Experiment group info to KPI dataframe (Age, Gender and Variation)


In [9]:
# Load datasets with required info

df_exp = pd.read_csv("../Data/Processed/df_final_experiment_clients_clean.csv") #variation info dataframe
df_demo = pd.read_csv("../Data/Processed/df_final_demo_clean.csv") #demographics dataframe


In [10]:
# Selecting only wanted columns from datasets

df_exp = df_exp[["client_id", "variation"]]
df_demo = df_demo[["client_id", "clnt_age", "gender", "clnt_tenure_yr"]]


In [11]:
# Left merge to the visit metrics csv (df_summary)

df_summary = df_summary.merge(df_exp, on="client_id", how="left")
df_summary = df_summary.merge(df_demo, on="client_id", how="left")


In [12]:
# NaN values on variation and gender replaced by "excluded"
df_summary["variation"] = df_summary["variation"].fillna("excluded")
df_summary["gender"] = df_summary["gender"].fillna("unknown")

# Client Age also has NaN values, replace with "unknown"

df_summary["clnt_age"] = df_summary["clnt_age"].fillna("unknown")

In [13]:
df_summary.to_csv("../Data/Processed/df_web_visit_metrics_final.csv", index=False)


In [14]:
df_summary.head()

,client_id,visitor_id,visit_id,steps_count,unique_steps,date,completion_status,drop_off_step,most_frequent_step,avg_time_per_step,...,time_in_start_sec,time_in_step_1_sec,time_in_step_2_sec,time_in_step_3_sec,total_time_sec,total_time,variation,clnt_age,gender,clnt_tenure_yr
0,169,201385055_71273495308,749567106_99161211863_557568,5,5,2017-04-12,1,None,none,0:42,...,9.0,46.0,94.0,64.0,213.0,3:33,excluded,47.5,Male,21.0
1,336,64757908_3400128256,649044751_80905125055_554468,2,1,2017-06-01,0,start,start,7:54,...,948.0,NaN,NaN,NaN,NaN,None,excluded,unknown,unknown,NaN
2,546,475037402_89828530214,731811517_9330176838_94847,5,5,2017-06-17,1,None,none,0:26,...,10.0,13.0,87.0,23.0,133.0,2:13,excluded,unknown,unknown,NaN
3,555,402506806_56087378777,637149525_38041617439_716659,5,5,2017-04-15,1,None,none,0:31,...,7.0,32.0,99.0,20.0,158.0,2:38,Test,29.5,Unknown,3.0
4,647,66758770_53988066587,40369564_40101682850_311847,5,5,2017-04-12,1,None,none,1:15,...,7.0,18.0,189.0,163.0,377.0,6:17,Test,57.5,Male,12.0


In [16]:
df_summary.shape

(159112, 23)

In [17]:
df_summary.columns

Index(['client_id', 'visitor_id', 'visit_id', 'steps_count', 'unique_steps',
       'date', 'completion_status', 'drop_off_step', 'most_frequent_step',
       'avg_time_per_step', 'backtrack_flag', 'backtrack_count',
       'time_in_confirm_sec', 'time_in_start_sec', 'time_in_step_1_sec',
       'time_in_step_2_sec', 'time_in_step_3_sec', 'total_time_sec',
       'total_time', 'variation', 'clnt_age', 'gender', 'clnt_tenure_yr'],
      dtype='object')

## Creating a DataFrame for only experiment participants


### Filter and create df using variation

In [18]:
df_exp_visits = df_summary[
    df_summary["variation"].isin(["Control", "Test"])
].copy() 

# this df is still visit level, with multiple rows per customer


In [20]:
df_exp_visits.head()

,client_id,visitor_id,visit_id,steps_count,unique_steps,date,completion_status,drop_off_step,most_frequent_step,avg_time_per_step,...,time_in_start_sec,time_in_step_1_sec,time_in_step_2_sec,time_in_step_3_sec,total_time_sec,total_time,variation,clnt_age,gender,clnt_tenure_yr
3,555,402506806_56087378777,637149525_38041617439_716659,5,5,2017-04-15,1,None,none,0:31,...,7.0,32.0,99.0,20.0,158.0,2:38,Test,29.5,Unknown,3.0
4,647,66758770_53988066587,40369564_40101682850_311847,5,5,2017-04-12,1,None,none,1:15,...,7.0,18.0,189.0,163.0,377.0,6:17,Test,57.5,Male,12.0
11,934,810392784_45004760546,7076463_57954418406_971348,4,1,2017-04-18,0,start,start,0:35,...,142.0,NaN,NaN,NaN,NaN,None,Test,51.0,Female,9.0
12,1028,42237450_62128060588,557292053_87239438319_391157,9,4,2017-04-08,0,step_1,step_1,0:59,...,49.0,94.0,284.0,111.0,NaN,None,Control,36.0,Male,12.0
14,1104,194240915_18158000533,543158812_46395476577_767725,1,1,2017-06-12,0,start,none,0:00,...,0.0,NaN,NaN,NaN,NaN,None,Control,48.0,Unknown,5.0


### Create a client level df

In [21]:
# Dataframe with all experiment participants with one row per client

# client-level aggregation (1 row per client)
df_kpi_clients = (
    df_exp_visits
    .sort_values("date")
    .groupby("client_id", as_index=False)
    .agg({
        "variation": "first",                 # Control or Test
        "completion_status": "max",            # 1 if completed at least once
        "total_time_sec": "min",               # first / fastest successful completion
        "backtrack_flag": "max",               # if any backtracking
        "backtrack_count": "sum",              # total backtracks across multiple visits
        "steps_count": "sum",                  # total steps across multiple attempts
        "unique_steps": "max",                 # max unique steps seen
        "clnt_age": "first",
        "gender": "first",
        "clnt_tenure_yr": "first"
    })
)

# keep time only for clients who completed
df_kpi_clients.loc[df_kpi_clients["completion_status"] != 1, "total_time_sec"] = np.nan


In [22]:
df_kpi_clients.head()

,client_id,variation,completion_status,total_time_sec,backtrack_flag,backtrack_count,steps_count,unique_steps,clnt_age,gender,clnt_tenure_yr
0,555,Test,1,158.0,0,0,5,5,29.5,Unknown,3.0
1,647,Test,1,377.0,0,0,5,5,57.5,Male,12.0
2,934,Test,0,NaN,0,0,4,1,51.0,Female,9.0
3,1028,Control,0,NaN,1,2,9,4,36.0,Male,12.0
4,1104,Control,0,NaN,0,0,2,1,48.0,Unknown,5.0


In [23]:
df_kpi_clients.shape

(50500, 11)

- Shape matches the shape of the experiment_clients dataframe